<a href="https://colab.research.google.com/github/0Nguyen0Cong0Tuan0/Road-Buddy-Challenge/blob/main/models/yolo_finetune_road_lane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLO11 Fine-tuning on Road Lane Dataset**

This notebook **fine-tunes YOLO11n and YOLO11l** on the Road Lane v2 dataset to detect **lane line types**.

**Dataset**

| Dataset | Classes | Focus |
|---------|---------|-------|
| Road Lane v2 | 6 | Lane line types |


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Setup & Installation**

In [13]:
!pip install ultralytics -q

In [14]:
import ultralytics
import os
import yaml
import time
from collections import Counter
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import shutil
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'models' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'traffic datasets'
MODELS_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'models'
RUNS_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'runs'

# Dataset paths
DATASETS = {
    'road_lane': {
        'path': DATA_DIR / 'Road Lane.v2i.yolo26',
        'yaml': DATA_DIR / 'Road Lane.v2i.yolo26' / 'data.yaml',
        'description': 'Road Lane v2 - Lane line types',
        'classes': ['divider-line', 'dotted-line', 'double-line',
                   'random-line', 'road-sign-line', 'solid-line']
    }
}

for name, info in DATASETS.items():
    exists = info['path'].exists()
    yaml_exists = info['yaml'].exists()
    print(f"{name}: {info['path'].name}")
    print(f"   YAML: {info['yaml'].name} ({'found' if yaml_exists else 'NOT FOUND'})")
    print(f"   Classes: {len(info['classes'])} - {info['classes'][:5]}...")

bdd100k: bdd100k
   YAML: data_yolo.yaml (found)
   Classes: 12 - ['car', 'truck', 'bus', 'train', 'person']...
road_lane: Road Lane.v2i.yolo26
   YAML: data.yaml (found)
   Classes: 6 - ['divider-line', 'dotted-line', 'double-line', 'random-line', 'road-sign-line']...


In [16]:
# Training configuration
TRAINING_CONFIG = {
    'epochs': 50,          # Number of training epochs
    'imgsz': 640,          # Image size
    'batch': 16,           # Batch size (reduce if GPU OOM)
    'patience': 10,        # Early stopping patience
    'device': DEVICE,      # Training device
    'workers': 4,          # Data loader workers
    'save': True,          # Save checkpoints
    'plots': True,         # Generate training plots
    'verbose': True,       # Verbose output
}


print("\nTraining Configuration")

for key, value in TRAINING_CONFIG.items():
    print(f"   {key}: {value}")


Training Configuration
   epochs: 50
   imgsz: 640
   batch: 16
   patience: 10
   device: cuda
   workers: 4
   save: True
   plots: True
   verbose: True


## **Fine-tuning on Road Lane Dataset**

### **Train YOLO11n on Road Lane**

In [19]:
model_n_lane = YOLO('yolo11n.pt')
model_n_lane.info()

YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs


(181, 2624080, 0, 6.614336)

In [20]:
print("Fine-tuning YOLO11n on Road Lane Dataset")

# Start training
results_n_lane = model_n_lane.train(
    data=str(DATASETS['road_lane']['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11n_road_lane',
    exist_ok=True,
    **TRAINING_CONFIG
)

Fine-tuning YOLO11n on Road Lane Dataset
Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/traffic datasets/Road Lane.v2i.yolo26/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolo11n_road_lane, nbs=64, nms=False, opset=None, op

### **Train YOLO11l on Road Lane**

In [21]:
model_l_lane = YOLO('yolo11l.pt')
model_l_lane.info()

YOLO11l summary: 357 layers, 25,372,160 parameters, 0 gradients, 87.6 GFLOPs


(357, 25372160, 0, 87.6134912)

In [22]:
# Reduce batch size for larger model
config_l = TRAINING_CONFIG.copy()
config_l['batch'] = 8

results_l_lane = model_l_lane.train(
    data=str(DATASETS['road_lane']['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11l_road_lane',
    exist_ok=True,
    **config_l
)

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/traffic datasets/Road Lane.v2i.yolo26/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolo11l_road_lane, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask